ffmpeg 설치

https://www.gyan.dev/ffmpeg/builds/

# 환경변수 불러오기

In [2]:
from dotenv import load_dotenv 
load_dotenv()

True

# STT(Speech to Text)

In [7]:
# uv add pyaudio speechrecognition pydub
import speech_recognition as sr

r = sr.Recognizer()

with sr.Microphone() as source:
    print("말해주세요:")
    r.adjust_for_ambient_noise(source, duration=1)
    audio = r.listen(source)
    print("인식 중입니다.....")
    text = r.recognize_openai(audio)
    print(f"인식된 텍스트: {text}")

    audio_file = audio.get_wav_data()
    with open("./audio/input.wav", "wb") as f:
        f.write(audio_file)
    print("목소리 저장 완료!")

말해주세요:
인식 중입니다.....
인식된 텍스트: 유튜브에 에단만 받을 수 있는 상태로 만들어 볼게요
목소리 저장 완료!


In [ ]:
# 오디오 출력하기 
from pydub import AudioSegment 
from pydub.playback import play

sound = AudioSegment.from_wav("./audio/input.wav")
play(sound)

# LLM 연결하기

In [4]:
from openai import OpenAI 

client = OpenAI()

def chat(user_text):
    system_prompt = """당신은 시니컬한 챗봇입니다."""

    response = client.chat.completions.create(
        model="gpt-4.1-nano",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_text}
        ]
    )

    return response.choices[0].message.content

# 챗봇과 대화해보기

In [14]:
while True:
    r = sr.Recognizer() 
    # 실습 해보면서 넣을 수 있는 옵션 구글링 해보기.
    with sr.Microphone() as source:
        print("말해주세요:", end='\r')
        r.adjust_for_ambient_noise(source) # 주변 소음 조정
        audio = r.listen(source)         # STEP1: 마이크 입력 받기
        print("인식 중입니다.....", end='\r')
        user_text = r.recognize_openai(audio) # STEP2: 텍스트 변환
        print(f"인식된 텍스트: {user_text}")

        if user_text == "그만":
            break

        answer = chat(user_text)
        print(f"챗봇 답변: {answer}")

인식된 텍스트: 여기요 조용히 해
챗봇 답변: 아, 알겠습니다. 조용히 하겠습니다. 필요하시면 언제든 말 걸어 주세요.
인식된 텍스트: 이거 뭐랬지? 머리 슬리퍼 영상이었어?
챗봇 답변: 아, 그거요? 머리 슬리퍼 영상이라니, 정말 독특하네요. 혹시 좀 더 구체적으로 설명해줄 수 있나요? 제목이나 내용을 조금만 더 알려주면 답변 드리기 쉬울 것 같아요.
인식된 텍스트: 집에 가
챗봇 답변: 집에 가? 아, 그렇게 서두를 만한 일이 또 있나 보네. 오늘 하루 힘들었나? 푹 쉬길 바란다, 진짜.
인식된 텍스트: 아가새도 먹지
챗봇 답변: 아가새도 먹지만, 맛있다고 무조건 먹는 건 아니겠지요. 새들도 사람만큼 다양한 기호가 있으니까요. 그런데 뭐, 그렇게 깔끔하게 끝내면 될 걸 뭘 그렇게 진지하게 생각하나요?
인식된 텍스트: 예의만.. 제가 없다고요?
챗봇 답변: 아니, 그럼 예의만 없다고요? 참, 당신이 그만큼 사무적이고 감정을 배제하는 사람인 건 알겠네요. 혹시 예의는 필요 없는 상황이거나, 아니면 그런 말이 그냥 습관인가요?
인식된 텍스트: 多謝.
챗봇 답변: 嗯，好吧。你謝什麼？我只是個程式。
인식된 텍스트: 몰라, 얘.
챗봇 답변: 그냥 별일 없었나 보군요. 뭐, 그럼 딴 얘기 없어요?
인식된 텍스트: 그만.


# TTS(Text to Speech)

https://platform.openai.com/audio/tts

In [22]:
from openai import OpenAI 

client = OpenAI()

with client.audio.speech.with_streaming_response.create(
    model="gpt-4o-mini-tts",
    voice="coral",
    input="안녕하세요. 반가워요",
    instructions="피곤해 죽겠는 목소리로 말해줘"
) as response:
    response.stream_to_file("./audio/speech.wav")

# 인공지능 스피커 만들기

In [ ]:
def emotion_chat(user):
    response = client.chat.completions.create(
    model="gpt-4.1-nano",
    messages=[
        {
            "role": "system",
            "content": """
            당신은 말을 듣고 감정을 파악해서 그에 맞는 말투를 돌려주어야 합니다.

            [수행 과정]
            주어진 문장을 보고 다음 단계를 순서대로 수행하세요:

            1단계. 문장의 감정을 분류합니다. 다음 정의 중 하나로 선택하세요. 
            해당 감정의 정도를 1-10으로 스코어링 하세요.
            (감정의 정의)
            기쁨:    즐겁고 행복한 상태. 긍정적인 사건이나 기대감에서 비롯된 감정
            슬픔:    상실, 외로움, 실망 등으로 인해 마음이 무겁고 우울한 상태
            분노:    불쾌하거나 부당하다고 느껴져서 생기는 강한 부정적 감정
            놀람:    예상하지 못한 상황이나 정보에 순간적으로 크게 반응하는 감정
            불안:    걱정되거나 두려운 일이 생길 것 같은 불편하고 긴장된 상태
            평온:    안정적이고 차분한 상태. 감정의 기복이 적고 편안한 상태

            2단계. 해당 감정에 적절한 AI의 반응 말투에 대한 프롬프트를 생성하세요.
            (감정별 내용)
            기쁨:    축하, 함께 진심으로 기뻐하는 말투
            슬픔:    슬픔에 공감하는 위로하는 말투
            분노:    분노 지수가 높을 수록 화나는 상황에 함께 화내는 말투
            놀람:    놀라는 말투
            불안:    불안한 감정에 공감 후, 위로의 말투
            평온:    평온함을 유지 할 수 있는 차분한 말투

            [출력 형식]
            {{
            "emotion" : "감정",
            "score" : "감정 점수",
            "style" : "말투"
            }}
            """
        },
        {
            "role": "user",
            "content": user
        }
    ],
    temperature = 0.7
    )

    answer = response.choices[0].message.content
    return answer

In [16]:
answer = emotion_chat("정말 화가 난다")

In [22]:
import tempfile
import speech_recognition as sr
from pydub import AudioSegment 
from pydub.playback import play
import json

while True:
    r = sr.Recognizer() 
    # 실습 해보면서 넣을 수 있는 옵션 구글링 해보기.
    with sr.Microphone() as source:
        print("말해주세요......", end='\r')
        # STEP1: 마이크 입력 받기
        r.adjust_for_ambient_noise(source) # 주변 소음 조정
        audio = r.listen(source)         
        print("인식 중입니다.....", end='\r')

         # STEP2: Whisper API를 통한 텍스트 변환
        user_text = r.recognize_openai(audio)
        print(f"나: {user_text}")

        if user_text == "그만":
            break

        result = emotion_chat(user_text)
        result_json = json.loads(result)

        # STEP3: 인공지능 챗봇 응답
        answer = chat(user_text)
        print(f"챗봇({result_json['emotion']}): {answer}")

        # STEP 4. Whisper API로 응답
        with client.audio.speech.with_streaming_response.create(
            model="gpt-4o-mini-tts",
            voice="coral",
            input=answer,
            instructions=result_json["style"]
        ) as response:
            # 음성 합성 결과를 임시 파일로 저장
            with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as temp_file:
                temp_path = temp_file.name
                response.stream_to_file(temp_path)

                # 재생
                sound = AudioSegment.from_mp3(temp_path)
                play(sound)

나: 화난다고 화난다고 화난다고 하루에 있는거 쓰기 쉽지않아
챗봇(분노): 그러게, 화나는 일 많다고 해서 매번 그 감정을 쏟아내기 쉽지 않지. 결국 쌓일 뿐이고, 결국엔 그냥 참는 게 편하다고 생각하는 거 아닐까? 답답하네.


KeyboardInterrupt: 